<a href="https://colab.research.google.com/github/j-hartmann/siebert/blob/main/SieBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **SieBERT: Leveraging Transfer Learning for Sentiment Analysis**



In [ ]:
# install Hugging Face's transformers and datasets libraries
!pip install transformers
!pip install datasets

In [8]:
# check GPU status
!nvidia-smi

Sun May  8 06:21:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
# check version of transformers
import transformers
print(transformers.__version__)

4.18.0


### **Example 1:** Applying SieBERT, a pretrained sentiment analysis model, with *3 lines of code*

In [10]:
from transformers import pipeline  # load pipeline() function from transformers library
sentiment_analysis = pipeline("sentiment-analysis", model="siebert/sentiment-roberta-large-english")  # load pretrained SieBERT model ("Sentiment in English")

Downloading:   0%|          | 0.00/687 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [11]:
sentiment_analysis("This is super helpful. I love it!")  # apply pretrained model to example sentence

[{'label': 'POSITIVE', 'score': 0.9988920092582703}]

### **Example 2:** Classifying multiple sentences using SieBERT

In [12]:
# load dependencies
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# specify path of pretrained model
checkpoint = "siebert/sentiment-roberta-large-english"  # SieBERT

# load pretrained tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

In [13]:
# provide 2 example sentences
sequences = ["This is amazing", "I don't think it's useless.", "I hate this!"]

# tokenize sequences
tokens = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

# predict with model
output = model(**tokens)

# transform logits to class labels
predictions = torch.nn.functional.softmax(output.logits, dim=-1)
confidences = predictions.max(1)[0].tolist()
classes = predictions.argmax(-1).tolist()
labels = pd.Series(classes).map(model.config.id2label)

In [14]:
# consolidate results
df = pd.DataFrame(list(zip(sequences, classes, labels, confidences)), columns=['text', 'class', 'class_label', 'confidence'])

# return dataframe
print(df)

                          text  class class_label  confidence
0              This is amazing      1    POSITIVE    0.998669
1  I don't think it's useless.      1    POSITIVE    0.991253
2                 I hate this!      0    NEGATIVE    0.999456


### **Example 3:** Fine-tuning SieBERT for multi-class sentiment analysis in a different domain

In [15]:
# load three-class sentiment data set from Hugging Face
from datasets import load_dataset
sentiment = load_dataset('sentiment140')  # source: https://huggingface.co/datasets/sentiment140/viewer/sentiment140/test
print(sentiment)

Generating train split:   0%|          | 0/1600000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/498 [00:00<?, ? examples/s]

Dataset sentiment140 downloaded and prepared to /root/.cache/huggingface/datasets/sentiment140/sentiment140/1.0.0/f81c014152931b776735658d8ae493b181927de002e706c4d5244ecb26376997. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'date', 'user', 'sentiment', 'query'],
        num_rows: 1600000
    })
    test: Dataset({
        features: ['text', 'date', 'user', 'sentiment', 'query'],
        num_rows: 498
    })
})


In [16]:
# print first row from training data split
print(sentiment['train'][0])

# count number of labels
NUM_LABELS = len(set(sentiment['test']['sentiment']))
print(set(sentiment['test']['sentiment']))
print(NUM_LABELS)

{'text': "@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D", 'date': 'Mon Apr 06 22:19:45 PDT 2009', 'user': '_TheSpecialOne_', 'sentiment': 0, 'query': 'NO_QUERY'}
{0, 2, 4}
3


In [17]:
# define preprocessing function
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [18]:
# tokenize dataset
tokenized_sentiment = sentiment.map(preprocess_function, batched=True)

# use dynamic padding
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/1600 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [19]:
# define evaluation metrics
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [20]:
# initialize pretrained model with updated classification head
model2 = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=NUM_LABELS, ignore_mismatched_sizes=True)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at siebert/sentiment-roberta-large-english and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([2, 1024]) in the checkpoint and torch.Size([3, 1024]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([3]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
# set number of epochs
NUM_EPOCHS = 1
NUM_EXAMPLES = 400

In [22]:
# rename label column
tokenized_sentiment = tokenized_sentiment.rename_column("sentiment", "label")

In [23]:
from datasets import ClassLabel, Value

# update labels
def update_labels(example):
  example['label'] = example['label'] / 2
  return example

tokenized_sentiment = tokenized_sentiment.map(update_labels)

new_features = tokenized_sentiment['test'].features.copy()
new_features["label"] = ClassLabel(names=['neg', 'neu', 'pos'])
tokenized_sentiment['test'] = tokenized_sentiment['test'].cast(new_features)

  0%|          | 0/1600000 [00:00<?, ?ex/s]

  0%|          | 0/498 [00:00<?, ?ex/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [24]:
# check features
tokenized_sentiment['test'].features

{'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'date': Value(dtype='string', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'label': ClassLabel(num_classes=3, names=['neg', 'neu', 'pos'], id=None),
 'query': Value(dtype='string', id=None),
 'text': Value(dtype='string', id=None),
 'user': Value(dtype='string', id=None)}

In [25]:
# train SieBERT
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=NUM_EPOCHS,
    weight_decay=0.01,
    evaluation_strategy="epoch",
)

trainer = Trainer(
    model=model2,
    args=training_args,
    train_dataset=tokenized_sentiment["test"].select(range(0,NUM_EXAMPLES)),  
    eval_dataset=tokenized_sentiment["test"].select(range(NUM_EXAMPLES,498)),
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: date, text, user, query. If date, text, user, query are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 400
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 25


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.921058,0.683673,0.687327,0.719561,0.733903


The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: date, text, user, query. If date, text, user, query are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 98
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=25, training_loss=0.7060818481445312, metrics={'train_runtime': 13.4413, 'train_samples_per_second': 29.759, 'train_steps_per_second': 1.86, 'total_flos': 32064371807904.0, 'train_loss': 0.7060818481445312, 'epoch': 1.0})

In [26]:
# store evaluations for SieBERT
siebert_eval = trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: date, text, user, query. If date, text, user, query are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 98
  Batch size = 16


In [27]:
# specify path of pretrained model
checkpoint = "roberta-large"  # RoBERTa-large

# load pretrained tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# initialize pretrained model with updated classification head
model3 = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=NUM_LABELS, ignore_mismatched_sizes=True)

Could not locate the tokenizer configuration file, will try to use the model config instead.
https://huggingface.co/roberta-large/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp4p8xxi0u


Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

storing https://huggingface.co/roberta-large/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/dea67b44b38d504f2523f3ddb6acb601b23d67bee52c942da336fa1283100990.94cae8b3a8dbab1d59b9d4827f7ce79e73124efa6bb970412cd503383a95f373
creating metadata file for /root/.cache/huggingface/transformers/dea67b44b38d504f2523f3ddb6acb601b23d67bee52c942da336fa1283100990.94cae8b3a8dbab1d59b9d4827f7ce79e73124efa6bb970412cd503383a95f373
loading configuration file https://huggingface.co/roberta-large/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/dea67b44b38d504f2523f3ddb6acb601b23d67bee52c942da336fa1283100990.94cae8b3a8dbab1d59b9d4827f7ce79e73124efa6bb970412cd503383a95f373
Model config RobertaConfig {
  "_name_or_path": "roberta-large",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

storing https://huggingface.co/roberta-large/resolve/main/vocab.json in cache at /root/.cache/huggingface/transformers/7c1ba2435b05451bc3b4da073c8dec9630b22024a65f6c41053caccf2880eb8f.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
creating metadata file for /root/.cache/huggingface/transformers/7c1ba2435b05451bc3b4da073c8dec9630b22024a65f6c41053caccf2880eb8f.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
https://huggingface.co/roberta-large/resolve/main/merges.txt not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpv5_2i35e


...

storing https://huggingface.co/roberta-large/resolve/main/merges.txt in cache at /root/.cache/huggingface/transformers/20b5a00a80e27ae9accbe25672aba42ad2d4d4cb2c4b9359b50ca8e34e107d6d.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
creating metadata file for /root/.cache/huggingface/transformers/20b5a00a80e27ae9accbe25672aba42ad2d4d4cb2c4b9359b50ca8e34e107d6d.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
https://huggingface.co/roberta-large/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpt79fjkd1


Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

storing https://huggingface.co/roberta-large/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/e16a2590deb9e6d73711d6e05bf27d832fa8c1162d807222e043ca650a556964.fc9576039592f026ad76a1c231b89aee8668488c671dfbe6616bab2ed298d730
creating metadata file for /root/.cache/huggingface/transformers/e16a2590deb9e6d73711d6e05bf27d832fa8c1162d807222e043ca650a556964.fc9576039592f026ad76a1c231b89aee8668488c671dfbe6616bab2ed298d730
loading file https://huggingface.co/roberta-large/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/7c1ba2435b05451bc3b4da073c8dec9630b22024a65f6c41053caccf2880eb8f.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
loading file https://huggingface.co/roberta-large/resolve/main/merges.txt from cache at /root/.cache/huggingface/transformers/20b5a00a80e27ae9accbe25672aba42ad2d4d4cb2c4b9359b50ca8e34e107d6d.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
loading file https://huggingface.co/robe

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

storing https://huggingface.co/roberta-large/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/8e36ec2f5052bec1e79e139b84c2c3089cb647694ba0f4f634fec7b8258f7c89.c43841d8c5cd23c435408295164cda9525270aa42cd0cc9200911570c0342352
creating metadata file for /root/.cache/huggingface/transformers/8e36ec2f5052bec1e79e139b84c2c3089cb647694ba0f4f634fec7b8258f7c89.c43841d8c5cd23c435408295164cda9525270aa42cd0cc9200911570c0342352
loading weights file https://huggingface.co/roberta-large/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/8e36ec2f5052bec1e79e139b84c2c3089cb647694ba0f4f634fec7b8258f7c89.c43841d8c5cd23c435408295164cda9525270aa42cd0cc9200911570c0342352
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head

In [28]:
# train RoBERTa
trainer = Trainer(
    model=model3,
    args=training_args,
    train_dataset=tokenized_sentiment["test"].select(range(0,NUM_EXAMPLES)),  
    eval_dataset=tokenized_sentiment["test"].select(range(NUM_EXAMPLES,498)),
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: date, text, user, query. If date, text, user, query are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 400
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 25


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,1.128973,0.295918,0.252930,0.194259,0.362773


The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: date, text, user, query. If date, text, user, query are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 98
  Batch size = 16
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=25, training_loss=1.1199967956542969, metrics={'train_runtime': 13.7375, 'train_samples_per_second': 29.117, 'train_steps_per_second': 1.82, 'total_flos': 32064371807904.0, 'train_loss': 1.1199967956542969, 'epoch': 1.0})

In [29]:
# store evaluations for RoBERTa
roberta_eval = trainer.evaluate()
models = ['SieBERT', 'RoBERTa']
accuracies = [siebert_eval['eval_accuracy'], roberta_eval['eval_accuracy']]
f1_scores = [siebert_eval['eval_f1'], roberta_eval['eval_f1']]

The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: date, text, user, query. If date, text, user, query are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 98
  Batch size = 16


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [30]:
# consolidate results
eval = pd.DataFrame(list(zip(models, accuracies, f1_scores)), columns=['model', 'accuracy', 'f1_score'])

# return dataframe
eval

,model,accuracy,f1_score
0,SieBERT,0.683673,0.687327
1,RoBERTa,0.295918,0.252930


Source: https://huggingface.co/
